In [1]:
import os
from pathlib import Path
from os import path

import numpy as np
import pandas as pd

# 1. Input data

In [2]:
cwd = os.getcwd()
root_dir = Path(cwd).parent

In [3]:
## Input data

# Individual data
data_IND_tot = pd.read_csv(path.join(root_dir,
                                    "data/individual_data.csv")).drop(['Unnamed: 0'],axis=1)

# Household data
data_HH_tot = pd.read_csv(path.join(root_dir,
                                    "data/household_data.csv")).drop(['Unnamed: 0'],axis=1)

# 2. Preprocess data

## 2.1. Set exposure variables and subsets

In [4]:
## Set exposure variables necessary for analysis: WASH facilities' characteristics + potential confounders

# Dummy dataframe
df = data_HH_tot.copy()


## Water services

# Recode access to "basic water" (following JMP definition) VS any other service type (including unimproved)
basic_wt_mask = ((df.DrinkingWater_Private.isin(['Piped_dwel','Piped_yard']))|
                 ((df.DrinkingWater_Private=='Piped_neigh')&(df.DrinkingWaterDist<=30))|
                 ((df.DrinkingWater_Public.isin(['Public_Tap','Public_Dispenser','Ground_tube']))&(df.DrinkingWaterDist<=30))|
                 ((df.DrinkingWater_Public=='Vendor')&(df.DrinkingWater_PublicVendor=='Vendor_Truck')&(df.DrinkingWaterDist<=30))
                )
df["BasicWater"] = 0
df["BasicWater"][basic_wt_mask] = 1
    
# Recode availability of drinking water
suffi_wt_mask = (df.DrinkingWaterAvailability=="N")
df["SufficientWt"] = np.nan
df["SufficientWt"][~df["DrinkingWaterAvailability"].isna()] = 0
df["SufficientWt"][suffi_wt_mask] = 1
    
# Recode location of water source: public X private/within premises/at a neighbor's
wt_loc_mask = (df.DrinkingWaterGroup=="Public")
df["PublicWtSource"] = np.nan
df["PublicWtSource"][~df["DrinkingWaterGroup"].isna()] = 0
df["PublicWtSource"][wt_loc_mask] = 1
    
# Recode water piped to premises X any other type
piped_wt_mask = ((df["DrinkingWaterGroup"]=="Private")&
                 (df["DrinkingWater_Private"].isin(["Piped_dwel","Piped_yard"])))
df["PipedWt"] = np.nan
df["PipedWt"][df["DrinkingWaterGroup"].isin(["Public","Private"])] = 0
df["PipedWt"][piped_wt_mask] = 1


## Sanitation services

# "Improved" sanitation (according to WHO-UNICEF's JMP) VS "unimproved"
isan_mask_abi = ((((df.City=='Abidjan')&
                   (df.ToiletFacilityTYPE=='DryOrCompost')&
                   (df.ToiletFacilityTYPE_Dry.isin(['Dry_ImprSlab', # improved pit latrine
                                                    'Dry_VIP'])))| # VIP: ventilated, improved pit latrine
                  ((df.City=='Abidjan')&
                   (df.ToiletFacilityTYPE=='Flush')&
                   (df.ToiletFacilityTYPE_Flush.isin(['Flush_piped',
                                                      'Flush_septic_tank',
                                                      'Flush_coveredPit']))))
                 &(df.ToiletROOF=='Y')) # many toilets in Abidjan did not have a roof
isan_mask_nai = ((((df.City=='Nairobi')&
                   (df.ToiletFacilityTYPE=='DryOrCompost')&
                   (df.ToiletFacilityTYPE_Dry.isin(['Dry_ImprSlab', # improved pit latrine
                                                    'Dry_VIP', # VIP: ventilated, improved pit latrine
                                                    'Dry_FreshLife'])))| # Sanergy's FreshLife toilet unit
                  ((df.City=='Nairobi')&
                   (df.ToiletFacilityTYPE=='Flush')&
                   (df.ToiletFacilityTYPE_Flush.isin(['Flush_piped',
                                                      'Flush_septic_tank',
                                                      'Flush_coveredPit'])))))
df["ImprvSan"] = np.nan
df["ImprvSan"][(~df.ToiletFacility.isna())&(~df.ToiletFacility.isin(['NoFacility_Nature']))] = 0
df["ImprvSan"][isan_mask_abi] = 1
df["ImprvSan"][isan_mask_nai] = 1

# Shared toilet
wc_share_mask = (df["ToiletFacilitySHARE"]=='Y') # toilet shared by more than one household
df["WCshared"] = np.nan
df["WCshared"][(~df["ToiletFacilitySHARE"].isna())] = 0
df["WCshared"][wc_share_mask] = 1

# Recode location of toilet (within premises X out of premises, excluding open defecation from analysis)
public_sel = ["Public","Neighb_YardPlot","Neighb_Dwelling"] # any WC located OUT OF PREMISES
wc_loc_mask = (df["ToiletFacility"].isin(public_sel))
df["WCoutprem"] = np.nan
df["WCoutprem"][(~df["ToiletFacility"].isna())&(df["ToiletFacility"]!="NoFacility_Nature")] = 0
df["WCoutprem"][wc_loc_mask] = 1

# Perceived safety to go to WC
wc_unsf_mask = ((df["ToiletFacilitySAFE"]=="During_Day")|
                (df["ToiletFacilitySAFE"]=="Unsafe")) # if safe only during the day, or never, toilet is considered unsafe
df["WCunsafe"] = np.nan
df["WCunsafe"][~df["ToiletFacilitySAFE"].isna()] = 0
df["WCunsafe"][wc_unsf_mask] = 1


## Hygiene services

# Access to "basic" hygiene (according to WHO-UNICEF's JMP)
bhyg_mask_abi = ((df.City=='Abidjan')&
                 (df["ObsHandWashWATER"]=='Water_OK')& # water available at the moment of the survey
                 (df["ObsHandWashSOAP"]=='SoapOrDeterg')& # soap (or equivalent) available at the moment of the survey
                 (df["ObsHandWashPLACE"].isin(['Obs_Fixed','Obs_Mobile'])) # presence of amenity to wash hands
                )
bhyg_mask_nai = ((df.City=='Nairobi')&
                 (df["ObsHandWashWATER"]=='Water_OK')& # water available at the moment of the survey
                 (df["ObsHandWashSOAP"]=='SoapOrDeterg')& # soap (or equivalent) available at the moment of the survey
                 (df["ObsHandWashPLACE"].isin(['Obs_Fixed'])) # presence of FIXED amenity to wash hands ('mobile' amenities were not reliable in Nairobi)
                )
df["BasicHyg"] = np.nan
df["BasicHyg"][(~df["ObsHandWashWATER"].isna())&(~df["ObsHandWashSOAP"].isna())&(~df["ObsHandWashPLACE"].isna())] = 0
df["BasicHyg"][bhyg_mask_abi] = 1
df["BasicHyg"][bhyg_mask_nai] = 1

# Presence of soap
soap_mask = (df["ObsHandWashSOAP"]=='SoapOrDeterg') # soap (or equivalent) available at the moment of the survey
df["HdHgSoap"] = np.nan
df["HdHgSoap"][(~df["ObsHandWashSOAP"].isna())] = 0
df["HdHgSoap"][soap_mask] = 1


## Housing conditions

# Recode precarious housing (based on dwelling's materials)
prec_h_mask = ((df["HHEXTWALLS"].isin(['Stone_mud','ReusedWoodOrBamboo','Metal','No_walls']))|
               (df["HHFLOOR"].isin(['Other']))|
               (df["HHROOF"].isin(['Rustic_metal','Rustic_toles_fibr','Rustic_planks','Rustic_plastic'])))
df["PrecHHMat"] = np.nan
df["PrecHHMat"][(~df["HHEXTWALLS"].isna())&(~df["HHFLOOR"].isna())&(~df["HHROOF"].isna())] = 0
df["PrecHHMat"][prec_h_mask] = 1

# Recode overcrowding (whether number of habitants per room > 3, following 'slum' definition in SDG 11)
src = data_IND_tot.copy()
hab_cnt_df = src[['KEY','PARENT_KEY']].groupby('PARENT_KEY').count().reset_index() # count habitants per household
hab_cnt_df = hab_cnt_df.rename(columns={"KEY": "Cnt_Hab"})
hab_cnt_df = hab_cnt_df.rename(columns={"PARENT_KEY": "KEY"})
df = df.merge(hab_cnt_df,how='left',on='KEY') # join variable with count of houshehold members
df['HHROOMS'][df['HHROOMS'].isin([111,999])] = np.nan # exclude invalid answers & outliers regarding rooms
df['HabPerRoom'] = df['Cnt_Hab']/df['HHROOMS']
hiden_mask = (df["HabPerRoom"]>3) # high density if number of household members per room > 3
df["HiDensity"] = np.nan
df["HiDensity"][~df["HabPerRoom"].isna()] = 0
df["HiDensity"][hiden_mask] = 1

# Recode cooking area (whether cooking is done inside dwelling or elsewhere)
cook_mask = (df["HHCOOKING"]=='House')# cooking done inside dwelling
df["CookInsd"] = np.nan
df["CookInsd"][~df["HHCOOKING"].isna()] = 0
df["CookInsd"][cook_mask] = 1


## Presence of children under-5

# Input: dummy dataframe
src_data = data_IND_tot.copy()
# Source data: get only variables of interest
src_cU5 = pd.DataFrame([src_data["PARENT_KEY"],src_data["Age"]]).transpose()
src_cU5 = src_cU5.rename(columns={"PARENT_KEY": "KEY"})
# New column: indicate presence of at least one child under-5
df["Child_U5"] = 0
lst_key_U5 = list(src_cU5["KEY"][src_cU5["Age"]<5]) # list unique IDs of households with child under 5
df["Child_U5"][df["KEY"].isin(lst_key_U5)] = 1


## Potential confounders

# Education level of heads of households
src = data_IND_tot[data_IND_tot.Relation_to_HH=='Head'][['PARENT_KEY',
                                                         'School_past']]# subset ed. level of HH
# heads of households with at least secondary education
src['SecEduHH'] = np.nan
src['SecEduHH'][src['School_past'].isin(['No_Edu',
                                         'Early_CdE',
                                         'Primary',
                                         'Coranic'])] = 0 # up to primary education
src['SecEduHH'][src['School_past'].isin(['Secondary',
                                         'Secondary_1',
                                         'Secondary_2',
                                         'High_Ed'])] = 1 # secondary or higher education
src = src[~src.SecEduHH.isna()]
# In case a same household has 2 heads, merge lines
src = src[['SecEduHH','PARENT_KEY']].groupby(by="PARENT_KEY").max().reset_index()
src = src.rename(columns={'PARENT_KEY':'KEY'})
# Attribute education level of heads of households
df = df.merge(src,on="KEY",how='left')

# Recode Sex of head of household (female = 1)
df['Sex_HH_F'] = np.nan
df['Sex_HH_F'][df['Sex_HH']=='M'] = 0
df['Sex_HH_F'][df['Sex_HH']=='F'] = 1

In [5]:
## Subsets

# Abidjan
df_abi = df[df['City']=='Abidjan'].copy()
print("N for general pop. in Abidjan:",df_abi.shape[0])

# Nairobi
df_nai = df[df['City']=='Nairobi'].copy()
print("N for general pop. in Nairobi:",df_nai.shape[0])

N for general pop. in Abidjan: 567
N for general pop. in Nairobi: 1147


# 3. Descriptive household data

## 3.1. Female respondents / household heads

In [6]:
## Sex of HH
var = 'Sex_HH_F'
name = 'Female respondents'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# View
df_table

Female respondents (total data counts, 0=No/1=Yes):
 
0.0    1070
1.0     644
Name: Sex_HH_F, dtype: int64
 
Female respondents (counts & percentage):


,site,total count,total outcomes,percentage
0,Azito,266,77,28.947368
1,Mabatini,576,292,50.694444
2,Vietnam,571,195,34.150613
3,Williamsville,301,80,26.578073


In [7]:
## Level of education of HH
var = 'SecEduHH'
name = 'Household heads with secondary education'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Household heads with secondary education (total data counts, 0=No/1=Yes):
 
1.0    797
0.0    742
Name: SecEduHH, dtype: int64
 
Household heads with secondary education (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,236,181,76.694915,±5.39%
1,Mabatini,528,230,43.560606,±4.23%
2,Vietnam,511,309,60.469667,±4.24%
3,Williamsville,264,77,29.166667,±5.48%


## 3.2 Surveyed households with children under five years

In [8]:
## Level of education of HH
var = 'Child_U5'
name = 'Households with children under-five'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
df_table

Households with children under-five (total data counts, 0=No/1=Yes):
 
0    1066
1     648
Name: Child_U5, dtype: int64
 
Households with children under-five (counts & percentage):


,site,total count,total outcomes,percentage
0,Azito,266,105,39.473684
1,Mabatini,576,189,32.812500
2,Vietnam,571,224,39.229422
3,Williamsville,301,130,43.189369


## 3.3. WASH

In [9]:
# Access to "basic" water services
var = 'BasicWater'
name = 'Households with access to "basic" water services'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households with access to "basic" water services (total data counts, 0=No/1=Yes):
 
1    1662
0      52
Name: BasicWater, dtype: int64
 
Households with access to "basic" water services (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,266,263,98.872180,±1.27%
1,Mabatini,576,538,93.402778,±2.03%
2,Vietnam,571,567,99.299475,±0.68%
3,Williamsville,301,294,97.674419,±1.7%


In [10]:
# Access to water in sufficient quantity
var = 'SufficientWt'
name = 'Households with sufficient quantities of water'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households with sufficient quantities of water (total data counts, 0=No/1=Yes):
 
0.0    915
1.0    796
Name: SufficientWt, dtype: int64
 
Households with sufficient quantities of water (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,265,142,53.584906,±6.0%
1,Mabatini,575,188,32.695652,±3.83%
2,Vietnam,570,232,40.701754,±4.03%
3,Williamsville,301,234,77.740864,±4.7%


In [11]:
# Access to water from a public source
var = 'PublicWtSource'
name = 'Households using water mainly from a public source'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households using water mainly from a public source (total data counts, 0=No/1=Yes):
 
1.0    925
0.0    787
Name: PublicWtSource, dtype: int64
 
Households using water mainly from a public source (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,266,0,0.000000,±0.0%
1,Mabatini,575,444,77.217391,±3.43%
2,Vietnam,570,474,83.157895,±3.07%
3,Williamsville,301,7,2.325581,±1.7%


In [12]:
# Access to water piped to premises
var = 'PipedWt'
name = 'Households using mainly water piped to premises'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households using mainly water piped to premises (total data counts, 0=No/1=Yes):
 
0.0    1091
1.0     618
Name: PipedWt, dtype: int64
 
Households using mainly water piped to premises (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,264,254,96.212121,±2.3%
1,Mabatini,574,70,12.195122,±2.68%
2,Vietnam,570,21,3.684211,±1.55%
3,Williamsville,301,273,90.697674,±3.28%


In [13]:
# Access to "improved" sanitation
var = 'ImprvSan'
name = 'Households with access to "improved" sanitation'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households with access to "improved" sanitation (total data counts, 0=No/1=Yes):
 
1.0    1097
0.0     608
Name: ImprvSan, dtype: int64
 
Households with access to "improved" sanitation (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,266,222,83.458647,±4.47%
1,Mabatini,568,297,52.288732,±4.11%
2,Vietnam,570,465,81.578947,±3.18%
3,Williamsville,301,113,37.541528,±5.47%


In [14]:
# Toilet with a roof (variable only available for Abidjan)

df_c = df[df.City=='Abidjan'].copy()

# Recode variable
df_c['ToiletROOF_B'] = np.nan
mask_wcroof = (df_c.ToiletROOF=='Y')
df_c['ToiletROOF_B'][~df_c['ToiletROOF'].isna()] = 0
df_c['ToiletROOF_B'][mask_wcroof] = 1

# Stats
var = 'ToiletROOF_B'
name = 'Households with a toilet covered by a roof (variable only available for Abidjan)'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df_c[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df_c[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df_c[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df_c[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households with a toilet covered by a roof (variable only available for Abidjan) (total data counts, 0=No/1=Yes):
 
1.0    426
0.0    138
Name: ToiletROOF_B, dtype: int64
 
Households with a toilet covered by a roof (variable only available for Abidjan) (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,264,249,94.318182,±2.79%
1,Williamsville,300,177,59.000000,±5.57%


In [15]:
# Use of shared sanitation
var = 'WCshared'
name = 'Households sharing a toilet'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households sharing a toilet (total data counts, 0=No/1=Yes):
 
1.0    1324
0.0     347
Name: WCshared, dtype: int64
 
Households sharing a toilet (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,266,41,15.413534,±4.34%
1,Mabatini,566,550,97.173145,±1.37%
2,Vietnam,539,528,97.959184,±1.19%
3,Williamsville,300,205,68.333333,±5.26%


In [16]:
# Use of toilets out of premises
var = 'WCoutprem'
name = 'Households using a toilet out of premises'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households using a toilet out of premises (total data counts, 0=No/1=Yes):
 
0.0    1363
1.0     342
Name: WCoutprem, dtype: int64
 
Households using a toilet out of premises (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,266,2,0.751880,±1.04%
1,Mabatini,568,313,55.105634,±4.09%
2,Vietnam,570,12,2.105263,±1.18%
3,Williamsville,301,15,4.983389,±2.46%


In [17]:
# Households feeling unsafe to use the toilet whenever needed
var = 'WCunsafe'
name = 'Households feeling unsafe to use the toilet whenever needed'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households feeling unsafe to use the toilet whenever needed (total data counts, 0=No/1=Yes):
 
0.0    1027
1.0     329
Name: WCunsafe, dtype: int64
 
Households feeling unsafe to use the toilet whenever needed (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,64,8,12.500000,±8.1%
1,Mabatini,522,244,46.743295,±4.28%
2,Vietnam,553,15,2.712477,±1.35%
3,Williamsville,217,62,28.571429,±6.01%


In [18]:
# Households with access to "basic" hygiene
var = 'BasicHyg'
name = 'Households with access to "basic" hygiene'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households with access to "basic" hygiene (total data counts, 0=No/1=Yes):
 
0.0    941
1.0    364
Name: BasicHyg, dtype: int64
 
Households with access to "basic" hygiene (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,246,157,63.821138,±6.0%
1,Mabatini,377,32,8.488064,±2.81%
2,Vietnam,384,19,4.947917,±2.17%
3,Williamsville,298,156,52.348993,±5.67%


In [19]:
# Households with proper product to wash hands
var = 'HdHgSoap'
name = 'Households with proper product to wash hands'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households with proper product to wash hands (total data counts, 0=No/1=Yes):
 
1.0    890
0.0    415
Name: HdHgSoap, dtype: int64
 
Households with proper product to wash hands (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,246,168,68.292683,±5.82%
1,Mabatini,377,271,71.883289,±4.54%
2,Vietnam,384,261,67.968750,±4.67%
3,Williamsville,298,190,63.758389,±5.46%


## 3.4 Housing conditions

In [20]:
# Households living in an improvised dwelling
var = 'PrecHHMat'
name = 'Households living in an improvised dwelling'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households living in an improvised dwelling (total data counts, 0=No/1=Yes):
 
1.0    1226
0.0     487
Name: PrecHHMat, dtype: int64
 
Households living in an improvised dwelling (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,266,43,16.165414,±4.42%
1,Mabatini,576,541,93.923611,±1.95%
2,Vietnam,570,564,98.947368,±0.84%
3,Williamsville,301,78,25.913621,±4.95%


In [21]:
# Households living in an overcrowded dwelling
var = 'HiDensity'
name = 'Households living in an overcrowded dwelling'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households living in an overcrowded dwelling (total data counts, 0=No/1=Yes):
 
0.0    1165
1.0     539
Name: HiDensity, dtype: int64
 
Households living in an overcrowded dwelling (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,265,36,13.584906,±4.13%
1,Mabatini,572,206,36.013986,±3.93%
2,Vietnam,567,195,34.391534,±3.91%
3,Williamsville,300,102,34.000000,±5.36%


In [22]:
# Households living in a dwelling with an indoor cooking space
var = 'CookInsd'
name = 'Households living in a dwelling with an indoor cooking space'
print(name+' (total data counts, 0=No/1=Yes):')
print(' ')
print(df[var].value_counts())
print(' ')
# Descriptive stats
print(name+' (counts & percentage):')
df_cnt = df[['Site',var]].dropna().groupby(['Site']).count().astype(int).reset_index()
df_sum = df[['Site',var]].dropna().groupby(['Site']).sum().astype(int).reset_index()
df_pc = df[['Site',var]].dropna().groupby(['Site']).mean().reset_index()
df_table = df_cnt.merge(df_sum,on='Site')
df_table = df_table.merge(df_pc,on='Site')
df_table.columns = ['site','total count','total outcomes','percentage']
df_table['total outcomes'] = df_table['total outcomes'].astype(int)
df_table['percentage'] = df_table['percentage']*100
# 95% CI
ci_list = []
for P,N in zip(df_table['percentage'].to_list(),df_table['total count'].to_list()):
    ci = round((1.96*(np.sqrt((P*(100-P))/N))),2)
    ci_list = ci_list+['±'+str(ci)+'%']
df_table['95% CI'] = ci_list
# View
df_table

Households living in a dwelling with an indoor cooking space (total data counts, 0=No/1=Yes):
 
1.0    1441
0.0     273
Name: CookInsd, dtype: int64
 
Households living in a dwelling with an indoor cooking space (counts & percentage):


,site,total count,total outcomes,percentage,95% CI
0,Azito,266,195,73.308271,±5.32%
1,Mabatini,576,558,96.875000,±1.42%
2,Vietnam,571,562,98.423818,±1.02%
3,Williamsville,301,126,41.860465,±5.57%
